In [3]:
from datetime import datetime, timedelta
import json
import matplotlib.pyplot as plt
import pandas as pd
import time
import tweepy

In [7]:
auth = tweepy.OAuthHandler(consumer_key = "xxxxxxxxxxxxxxxxxxx",
                           consumer_secret = "xxxxxxxxxxxxxxxxxxx")
auth.set_access_token("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                      "xxxxxxxxxxxxxxxxxxx")

api = tweepy.API(auth)

current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
cutoff_time = (datetime.utcnow() - timedelta(days = 1)).strftime("%Y-%m-%d %H:%M:%S")

In [19]:
# Set the users of interest.
users = ['nytimes',
        'thesun',
        'thetimes',
        'ap',
        'cnn',
        'bbcnews',
        'cnet',
        'msnuk',
        'telegraph',
        'usatoday',
        'wsj',
        'washingtonpost',
        'bostonglobe',
        'newscomauhq',
        'skynews',
        'sfgate',
        'ajenglish',
        'independent',
        'guardian',
        'latimes',
        'reutersagency',
        'abc',
        'business',
        'bw',
        'time']

In [184]:
print("Getting tweets...")

tweets_data = []

# For each user, get the most recent tweets and store them in a dataframe.
for user in range(len(users)):
    user = api.get_user(users[user])
    recent_tweets = api.user_timeline(user_id = user.id)
    for tweet in range(len(recent_tweets)):
        json_str = json.dumps(recent_tweets[tweet]._json)
        tweet = json.loads(json_str)
        tweets_data.append(tweet)

print("Done.")

Getting tweets...
Done.


In [185]:
tweets = pd.DataFrame()

# Function to get the URL of a tweet even if it's a retweet.
def get_url(tweet):
    return("https://twitter.com/" + tweet['user']['screen_name'] + "/status/" + tweet['id_str'])

tweets['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['utc_offset'] = list(map(lambda tweet: tweet['user']['utc_offset'], tweets_data))
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['tweet_url'] = list(map(get_url, tweets_data))
tweets['screen_name'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['name'] = list(map(lambda tweet: tweet['user']['name'], tweets_data))
tweets['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
tweets['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))

In [180]:
#tweets.to_csv("../data/old_tweets.csv", index = False, index_label = False)

In [187]:
old_tweets = pd.read_csv("../data/old_tweets.csv")
tweets = tweets.append(old_tweets, ignore_index = True)
tweets = tweets.drop_duplicates(subset = 'tweet_url')
tweets = tweets.reset_index(drop = True)
recent_tweets = tweets

In [188]:
# Function to convert the time of the tweets to a proper datetime.
def tweet_time_to_utc(tweet):
    newtime = datetime.strptime(recent_tweets['created_at'][tweet], '%a %b %d %H:%M:%S +0000 %Y') + timedelta(seconds = int(recent_tweets['utc_offset'][tweet]))
    return(newtime)

recent_tweets['utc_created_at'] = list(map(tweet_time_to_utc, list(range(len(recent_tweets)))))
recent_tweets = recent_tweets[tweets['utc_created_at'] > cutoff_time]
recent_tweets = recent_tweets.reset_index(drop = True)
recent_tweets.drop('utc_created_at', 1).to_csv("../data/old_tweets.csv", index = False)

In [189]:
#pd.read_csv("../data/old_tweets.csv")
#for i in range(len(recent_tweets)):
#    print(i)
recent_tweets

,created_at,utc_offset,text,tweet_url,screen_name,name,retweet_count,favorite_count,utc_created_at
0,Sat May 20 22:49:49 +0000 2017,-14400,Chris Cornell sang as if he were bearing the w...,https://twitter.com/nytimes/status/86606344223...,nytimes,The New York Times,19,55,2017-05-20 18:49:49
1,Sat May 20 22:32:38 +0000 2017,-14400,"Where is North Korea? Here are guesses from 1,...",https://twitter.com/nytimes/status/86605911790...,nytimes,The New York Times,167,161,2017-05-20 18:32:38
2,Sat May 20 22:17:04 +0000 2017,-14400,Robert De Niro gives one of his best screen pe...,https://twitter.com/nytimes/status/86605519853...,nytimes,The New York Times,242,744,2017-05-20 18:17:04
3,Sat May 20 22:04:08 +0000 2017,-14400,The answers about what is happening in Antarct...,https://twitter.com/nytimes/status/86605194191...,nytimes,The New York Times,445,505,2017-05-20 18:04:08
4,Sat May 20 21:50:05 +0000 2017,-14400,"The prospect that 58,000 Haitians in the U.S. ...",https://twitter.com/nytimes/status/86604840851...,nytimes,The New York Times,214,267,2017-05-20 17:50:05
5,Sat May 20 21:38:07 +0000 2017,-14400,Regulatory rollbacks under EPA head Scott Prui...,https://twitter.com/nytimes/status/86604539623...,nytimes,The New York Times,192,133,2017-05-20 17:38:07
6,Sat May 20 21:24:05 +0000 2017,-14400,"How should Christian schools, which advocate a...",https://twitter.com/nytimes/status/86604186633...,nytimes,The New York Times,176,306,2017-05-20 17:24:05
7,Sat May 20 21:10:05 +0000 2017,-14400,Members of Trump's team suggested that he calm...,https://twitter.com/nytimes/status/86603834165...,nytimes,The New York Times,283,581,2017-05-20 17:10:05
8,Sat May 20 20:56:06 +0000 2017,-14400,A look at Triple Crown winner American Pharoah...,https://twitter.com/nytimes/status/86603482230...,nytimes,The New York Times,95,246,2017-05-20 16:56:06
9,Sat May 20 20:42:06 +0000 2017,-14400,Our top 10 comments of the week https://t.co/K...,https://twitter.com/nytimes/status/86603129887...,nytimes,The New York Times,159,618,2017-05-20 16:42:06


In [32]:
# old_tweets.to_csv("")
# time_of_most_recent_notification.to_txt("")
# new_notifications.to_csv("")
old_tweets

,Unnamed: 0,created_at,utc_offset,text,url,screen_name,name,retweet_count,favorite_count,utc_created_at
0,0,Sat May 20 21:38:07 +0000 2017,-14400,Regulatory rollbacks under EPA head Scott Prui...,https://twitter.com/nytimes/status/86604539623...,nytimes,The New York Times,74,60,2017-05-20 17:38:07
1,1,Sat May 20 21:24:05 +0000 2017,-14400,"How should Christian schools, which advocate a...",https://twitter.com/nytimes/status/86604186633...,nytimes,The New York Times,99,161,2017-05-20 17:24:05
2,2,Sat May 20 21:10:05 +0000 2017,-14400,Members of Trump's team suggested that he calm...,https://twitter.com/nytimes/status/86603834165...,nytimes,The New York Times,165,355,2017-05-20 17:10:05
3,3,Sat May 20 20:56:06 +0000 2017,-14400,A look at Triple Crown winner American Pharoah...,https://twitter.com/nytimes/status/86603482230...,nytimes,The New York Times,82,198,2017-05-20 16:56:06
4,4,Sat May 20 20:42:06 +0000 2017,-14400,Our top 10 comments of the week https://t.co/K...,https://twitter.com/nytimes/status/86603129887...,nytimes,The New York Times,139,529,2017-05-20 16:42:06
5,5,Sat May 20 20:15:04 +0000 2017,-14400,Brazilian president Michel Temer made it clear...,https://twitter.com/nytimes/status/86602449683...,nytimes,The New York Times,176,204,2017-05-20 16:15:04
6,6,Sat May 20 20:02:02 +0000 2017,-14400,Turkish NBA player Enes Kantor said he was det...,https://twitter.com/nytimes/status/86602121753...,nytimes,The New York Times,357,476,2017-05-20 16:02:02
7,7,Sat May 20 19:50:05 +0000 2017,-14400,“That doesn’t sound legal.” Nike’s Oregon Proj...,https://twitter.com/nytimes/status/86601820991...,nytimes,The New York Times,87,158,2017-05-20 15:50:05
8,8,Sat May 20 19:36:03 +0000 2017,-14400,-Louis C.K.'s odd accent\n-Aziz Ansari hosts\n...,https://twitter.com/nytimes/status/86601467842...,nytimes,The New York Times,160,656,2017-05-20 15:36:03
9,9,Sat May 20 19:22:04 +0000 2017,-14400,RT @nytopinion: President Trump is showing tha...,https://twitter.com/nytimes/status/86601115985...,nytimes,The New York Times,60,0,2017-05-20 15:22:04
